#Data Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langdetect

In [2]:
import re
import nltk
import pandas as pd
#from langdetect import detect, DetectorFactory
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
#import langdetect
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#read .txt file and convert it to dataframe
file_path='/content/drive/MyDrive/booksummaries.txt'
fields=['ID', 'Tag', 'Title', 'Author', 'Year', 'Genre', 'Description']
data_frame = pd.read_csv(file_path, sep='\t', header=None, names=fields)

In [4]:
#cleaning data frame from missing value:
data_frame_cleaned = data_frame.dropna(axis=1)
#remove some Description that has small number of characters and their Description is missing
data_frame_cleaned['len_of_Description']=data_frame_cleaned['Description'].apply(len)
data_frame_cleaned=data_frame_cleaned[data_frame_cleaned['len_of_Description']>250]
#sort Description baseed on len_of_Description parameter
data_frame_cleaned=data_frame_cleaned.sort_values(by=['len_of_Description'], ascending=True)

<ipython-input-4-04e103ad82f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame_cleaned['len_of_Description']=data_frame_cleaned['Description'].apply(len)


In [5]:
#Define some function for preprocessing the text
# Set seed to make results reproducible
#DetectorFactory.seed = 0

# def detect_language(text):
#     try:
#         return langdetect.detect(text)
#     except langdetect.lang_detect_exception.LangDetectException:
#         return None

# def find_non_english_sentences(text):
#     sentences = sent_tokenize(text)
#     non_english_sentences = [sentence for sentence in sentences if detect_language(sentence) != 'en']
#     flag=True if len(non_english_sentences)!=0 else False
#     return flag,non_english_sentences


def preprocess_text(text):
  # remove urls
  url_pattern = r'https?://\S+|www\.\S+'
  text = re.sub(url_pattern, '', text)

  # flag,non_english_sentences=find_non_english_sentences(text)
  # if flag:
  #   for non_english_sentence in non_english_sentences:
  #     text=re.sub(non_english_sentence,'',text)

  #remove special characters ---> ===
  text=re.sub(r'===','',text)

  return text

In [6]:
data_frame_cleaned['clean_Description']=data_frame_cleaned['Description'].apply(preprocess_text)

In [7]:
data_frame_cleaned.head()

,ID,Tag,Title,Description,len_of_Description,clean_Description
6549,5408961,/m/0dkk4_,Zamper,"The Doctor and his companions, separated from...",251,"The Doctor and his companions, separated from..."
13243,20297155,/m/04zxt3s,Blackthorn Winter,"Juliana, a fifteen-year old girl, moves with ...",251,"Juliana, a fifteen-year old girl, moves with ..."
10895,13733151,/m/03cgkpf,Rebekah,Rebekah follows the story of Isaac through th...,251,Rebekah follows the story of Isaac through th...
12925,19433109,/m/04n2tm1,Vaporetto 13: A Novel,Jack Squire is a Currency Trader on assignmen...,251,Jack Squire is a Currency Trader on assignmen...
13860,22286122,/m/05q4vyz,Heroes of Tobruk,The story follows Peter Fullerton and Tony Ca...,252,The story follows Peter Fullerton and Tony Ca...


NPL Componet(text summarization)

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [8]:
# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model.to('cuda')

In [ ]:
def add_summarize(text):
  text=str("summarize: " + text)
  return text

batch_size=256
data_numbers=len(data_frame_cleaned)
process_num=math.ceil(data_numbers/batch_size)


output_path='/content/drive/MyDrive/output_path.csv'
output_dict={'name':[],'summary':[]}


for i in range(process_num):
  #put data in a batch
  texts=list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['Description'].apply(add_summarize))

  #set min length of summary
  maxs=max(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['len_of_Description']))
  mins=min(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['len_of_Description']))
  min_len=min(500,int(0.2*mins))


  # Encode the text and generate summary
  inputs = tokenizer(texts, return_tensors="pt", max_length=maxs, padding="longest", truncation=True)
  #inputs = {k: v.cuda() for k, v in inputs.items()}
  input_x=inputs['input_ids'].cuda()



  # # Generate summaries
  summary_ids = model.generate(input_x, max_length=maxs, min_length=min_len, length_penalty=2.0, num_beams=4, early_stopping=True)
  summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]

  #save the output
  output_dict['name'].extend(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['Title']))
  output_dict['summary'].extend(summaries)
  temp_output=pd.DataFrame(output_dict)
  temp_output.to_csv(output_path)

  print(i)

